In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import lead , col , broadcast , collect_set ,size, array_contains

class Transformer:
    def __init__(self):
        pass

    def transform(self, inputDfs):
        pass


class AirpodsAfterIphoneTransformer(Transformer):  # Defined outside Transformer
    def transform(self, inputDfs):
        """
        Customers who have bought AirPods after buying the iPhone
        """
        transactionInputDf = inputDfs.get("transactionInputDf")
        print("transactionInputDf in transform ")
        transactionInputDf.show()

        # Define a window specification

        windowSpec = Window.partitionBy("customer_id").orderBy("transaction_date")

        # by default it is ascending 
        # Add a column with the next product name for each transaction

        transformedDF = transactionInputDf.withColumn(
            "next_product_name",
            lead("product_name").over(windowSpec)
        )

        print("AirPods after buying the iPhone:")
        transformedDF.orderBy("customer_id", "transaction_date", "product_name").show()

        filteredDf = transformedDF.filter(( col("product_name") =="iPhone") & (col("next_product_name") == "AirPods"))
            
        filteredDf.orderBy("customer_id", "transaction_date", "product_name").show()
        
        customerInputDf = inputDfs.get("customerInputDf")
         
        joinDf = customerInputDf.join(
            broadcast(filteredDf),
            "customer_id"
            )
        
        print("joined DF")
        joinDf.select("customer_id","customer_name","location").show()
        return joinDf

class OnlyAirpodsAndIphone(Transformer):
     def transform(self, inputDfs):
        """
        customerr who have bough only iphone and airpods nothing else 
        """
        transactionInputDf = inputDfs.get("transactionInputDf")
        print("transactionInputDf in transform ")

        groupedDF = transactionInputDf.groupBy("customer_id").agg(
        collect_set("product_name").alias("products")
        )

        print("Grouped DF")
        groupedDF.show()

        filteredDf = groupedDF.filter(
        (array_contains(col("products"), "iPhone")) &
        (array_contains(col("products"), "AirPods")) &
        (size(col("products")) == 2)
        ) 

        print ("only Airpods and iphone ")
        filteredDf.show()
        # return filteredDf
        customerInputDf = inputDfs.get("customerInputDf")
         
        joinDf = customerInputDf.join(
        broadcast(filteredDf),
            "customer_id"
        )
        
        print("joined DF")
        joinDf.select("customer_id","customer_name","location").show()
        return joinDf


